# Tutorial Step 1: Download GWOSC Data

This tutorial will show you how to download data from the [GWOSC website](https://gwosc.org).

## Browse available data sets

Go to the [GWOSC website](https://gwosc.org/) and open "Get Data" in the menu bar and then click on "Download".
The [Observatory Data Sets](https://gwosc.org/data/) page will display the list of data sets.

## Use timelines to show times of available data

Data for the LVK detectors is only publicly available during times when the detectors were operating under normal conditions, labeled with the DATA flag (i.e. DATA is available at this time).

The GWOSC Timelines provide a convenient, graphical tool for discovering times when detectors were collecting data.

Let's start by looking at the DATA Timeline for H1 during the Second Part of the Third Observing Run (O3b):
  - On the [Observatory Data Sets](https://gwosc.org/data/) page, find the "O3b" Data Release.
  - Click the [Timeline icon](https://gwosc.org/timeline/show/O3b_16KHZ_R1/H1_DATA*L1_DATA*V1_DATA/1256655618/12708000/). You should see plots like this: ![timeline](./timeline.png)

Here are some notes to help understand the GWOSC Timelines:
  - The label on the far left indicates which instrument is represented in the plot. In this example, H1 corresponds to the "Hanford One" detector.
  - The curve shows if the detector was collecting data at this particular time: a vertical bar indicates that the detector collects data while the absence of a bar indicates that the detector was not collecting data.
  - This data set spans the Second Part of the Third Observing Run (O3b) i.e. from 2019-11-01T15:00:00 UTC (GPS=1256655618) to 2020-03-27T17:00:00 UTC (GPS=1269363618).
  - You can use your mouse to zoom and pan the graphs and download the figure as an image on your computer by clicking on the "..." icon in the top-right corner of the plot.

## Download data files from the data archive

Now that we've got some idea when data is available, let's get the actual data files.
We'll try to find data for the month of January 2020.
We can query the O3b archive for the data we want:

  - Click the "Get Data" link in the menu bar and then on "Download".
  - Under "O3b Data Release", click the "4 kHz Data" icon:

    - Use the radio buttons to select "H1".
    - In the web form, enter the dates 2020-01-01T00:00:00 UTC (GPS=1261872018) and 2020-01-31T23:59:59 UTC (GPS=1264550417).
    - Then, click the continue button.

As you can see, we use the GPS time.
To find the GPS time of a specific UTC time (or vice-versa), you can use the [time conversion tool](https://gwosc.org/gps/).

This will query the database for data files between the entered dates.
You should see a list that looks like this: ![archive content](./archive_content.png)

Each line of the table corresponds to a data file (or tile) covering 4096 seconds of calendar time.
A given instrument may be up or down for any fraction of that time, and the far right column shows what percentage of the 4096 seconds contains science mode data.

Let's download one data file, which we'll use for the rest of this tutorial.
Since we want a file with mostly Science Mode data, let's download the file that starts at GPS time 1264312320.

To download the file, just click the link in the column under the heading "HDF5".
The downloaded file should be named `H-H1_GWOSC_O3b_4KHZ_R1-1264312320-4096.hdf5`.

<div class="alert alert-block alert-warning">
<div><b> &#9888 Warning</b></div>
    After the download completes, move the file in the directory where the notebooks are stored.
    If you use Binder to run the notebooks, upload this file to the Binder instance. 
</div>